CLASSIFIER TRAINED WITH ***ESTIMATE PARAMETERS***

* Create a sklearn KNN classifier and wrap it with SecML
* Create a SecML wrapper for the newly created classifier
* Fit the classifier on the MNIST dataset using SecML
* Compute the accuracy on the test set

In [1]:
try:
  import secml
except ImportError:
  %pip install git+https://github.com/pralab/secml

In [2]:
# 1. Data importation

n_tr = 3000
n_ts = 1000

from secml.data.loader import CDataLoaderMNIST

loader = CDataLoaderMNIST()
tr = loader.load('training')
ts = loader.load('testing', num_samples=n_ts)

tr.X /= 255
ts.X /= 255

In [3]:
# 2. Select a classifier. In this particular example: KNN classifier

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

In [4]:
# 3. SecML wrapper for the newly created classifier

from secml.ml.classifiers import CClassifierSkLearn
secml_model = CClassifierSkLearn(knn)

In [5]:
# 4. Select a metric to train our classifier
from secml.ml.peval.metrics import CMetricAccuracy
metric = CMetricAccuracy()

# Create a 3-Fold data splitter
from secml.data.splitter import CDataSplitterKFold
splitter = CDataSplitterKFold(num_folds=2)

# Select the parameters of our classifier that we want to prove
n_neighbors = {'n_neighbors': [7,9]}

In [ ]:
# 5.Select and set the best training parameters for the classifier
print("Estimating the best training parameters...")

best_nneigh = secml_model.estimate_parameters(
    dataset=tr,
    parameters=n_neighbors,
    splitter=splitter,
    metric=metric,
    perf_evaluator='xval'
)

Estimating the best training parameters...


In [ ]:
# We finally train our model with the best found parameters
best_nneigh.fit(tr.X, tr.Y)

In [ ]:
# 4. Accuracy

predictions = secml_model.predict(ts.X)

from secml.ml.peval.metrics import CMetricAccuracy
metric = CMetricAccuracy()

accuracy = metric.performance_score(y_true = ts.Y , y_pred = predictions)
print('Accuracy on the test set is', accuracy)

---------------------------------------------------------------

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from secml.data.loader import CDataLoaderMNIST

# Load the MNIST dataset
n_tr = 3000
n_ts = 1000

loader = CDataLoaderMNIST()
tr = loader.load('training')
ts = loader.load('testing', num_samples=n_ts)

# Define the KNN classifier
knn_classifier = KNeighborsClassifier()

# Define the parameter grid for the grid search
param_grid = {'n_neighbors': [3, 5, 7, 9, 11]}

# Create the GridSearchCV object
grid_search = GridSearchCV(knn_classifier, param_grid, cv=5, scoring='accuracy')

# Fit the grid search on the training data
import numpy as np

x_train = tr.X.tondarray()
y_train = tr.Y.tondarray()

grid_search.fit(x_train, y_train)

# Get the best hyperparameters and the corresponding accuracy
best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_

print(f"Best n_neighbors: {best_params['n_neighbors']}")
print(f"Accuracy with the best n_neighbors: {best_accuracy:.2%}")


